# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [1]:
import pandas as pd
import numpy as np
from modules.data_preprocessing import clean_raw, loader
from modules.weather import harvest_weather_data, clean_weather, light_clean
pd.options.mode.chained_assignment = None
#clean_raw(path='../data/raw_regression_data/*',save_path='../data/preprocessed_data_regression/',overwrite=True)
jan = pd.concat([pd.read_feather('../data/preprocessed_data_regression/delayed_01.feather'),pd.read_feather('../data/preprocessed_data_regression/delayed_02.feather')])

In [94]:
#clean_raw(path='../data/raw_regression_data/*',save_path='../data/preprocessed_data_regression/',overwrite=True,day_of_week=False)
jan = pd.concat([pd.read_feather('../data/preprocessed_data_regression/delayed_01.feather'),pd.read_feather('../data/preprocessed_data_regression/delayed_02.feather')])
#arr_weather = harvest_weather_data(jan[['dest_city_name','fl_date']],save=True,prefix='regression_inner_',rotate=True)
arr_weather = pd.read_csv('../data/weather/regression_inner_weather_data.csv')
dep_weather = pd.read_csv('../data/weather/regression_inner_dep_weather_data.csv')
arr_weather.columns = ['Unnamed: 0','arr_weather']
arr_weather = light_clean(arr_weather.arr_weather)
dep_weather = light_clean(dep_weather.weather)
dep_weather.rename('dep_weather', inplace=True)
jan = pd.concat([jan.reset_index(drop=True),arr_weather.reset_index(drop=True), dep_weather.reset_index(drop=True)],axis=1)
#jan.drop('Unnamed: 0', axis=1, inplace=True)
carrier_stats = pd.read_csv('../data/raw_regression_passenger/carrier_passenger_stats.csv')
jan = pd.merge(jan,carrier_stats,left_on='op_unique_carrier',right_on='unique_carrier')
jan.drop('unique_carrier',axis=1,inplace=True)
jan['rush_hour'] = jan.crs_arr_time.transform(lambda x: (x > 5) and (x < 20)).map({True:1,False:0})
jan['rush_hour_dep'] = jan.crs_dep_time.transform(lambda x: (x > 5) and (x < 20)).map({True:1,False:0})
origin_delay = jan.groupby(by='origin').arr_delay.mean()
dest_delay = jan.groupby(by='dest').total_delay.mean()
jan = pd.merge(jan,origin_delay,left_on='origin',right_on=origin_delay.index)
jan = pd.merge(jan,dest_delay,left_on='dest',right_on=dest_delay.index)
flight_time = pd.read_csv('../data/raw_regression_passenger/carrier_avg_flight_time.csv')
jan = pd.merge(jan,flight_time,left_on='op_unique_carrier',right_on='op_unique_carrier')
jan.columns = ['fl_date', 'op_unique_carrier', 'origin', 'dest', 'origin_city_name',
       'dest_city_name', 'crs_dep_time', 'crs_arr_time', 'crs_elapsed_time',
       'arr_delay', 'dep_delay', 'arr_weather', 'dep_weather',
       'avg_occupancy', 'avg_passengers', 'avg_seats', 'rush_hour',
       'rush_hour_dep', 'arr_delay_avg', 'dest_delay_avg', 'avg_carrier_speed']

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [96]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

one_hot = OneHotEncoder()

X = jan[['fl_date', 'op_unique_carrier', 'origin', 'dest','crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'arr_weather', 'dep_weather',
       'avg_occupancy', 'avg_seats', 'avg_passengers','rush_hour','rush_hour_dep',
       'arr_delay_avg', 'dest_delay_avg', 'avg_carrier_speed']]
       
one_hot.fit(X[['fl_date','op_unique_carrier','origin','dest', 'arr_weather', 'dep_weather']])
encoded_X = one_hot.transform(X[['fl_date','op_unique_carrier','origin','dest', 'arr_weather', 'dep_weather']]).toarray()
encoded_df = pd.DataFrame(encoded_X, columns=one_hot.get_feature_names(['fl_date','op_unique_carrier','origin','dest', 'arr_weather', 'dep_weather']))
df = pd.concat([X.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)
df.drop(['op_unique_carrier','origin','dest','fl_date', 'arr_weather', 'dep_weather','avg_seats'],inplace=True,axis=1)
y = jan.arr_delay

x_train,x_test,y_train,y_test=train_test_split(df,y,test_size=0.2)

c:\Users\malco\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [68]:
'''import pickle
pickle.dump(rand_forest, open('first_forest.pkl', 'wb'))'''

In [ ]:
import pickle
loaded_model = pickle.load(open('first_forest.pkl' , 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)

In [151]:
import xgboost as xgb

params = {
    "max_depth": 13,
    "eta": 0.2,
    'n_estimators':150
}

xgb_reg = xgb.XGBRegressor(**params)

xgb_reg.fit(x_train,y_train)

y_pred = xgb_reg.predict(x_test)

In [160]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
model = pickle.load(open('xgboost_MSE_580.pkl', 'rb'))
y_pred = model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred,squared=False)
r2 = r2_score(y_test,y_pred)
mea = mean_absolute_error(y_test,y_pred)
print("Mean squared error: ", mse)
print("root mean squared error: ", rmse)
print('R squared:',r2)
print('Mean Absolute Error:',mea)

Mean squared error:  580.8074774228031
root mean squared error:  24.099947664316684
R squared: 0.3518731304956587
Mean Absolute Error: 15.960658069836947


This model has a max depth of 13, a learning rate of 0.2, and 150 trees.

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
param_grid = {
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1],
    'n_estimators': [50, 100]
}

xgb_reg = xgb.XGBRegressor()

grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(x_train, y_train)

print("Best hyperparameters: ", grid_search.best_params_)
print("MSE: ", -grid_search.best_score_)

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.